Nom Etudiant 1: Malek Bouzarkouna 28706508
<br>
Nom Etudiant 2: Sevag BABOYAN 3521887

# TME 2.2 : Median String


## Partie A : Recheche de pattern (motifs) en permettant des variations

Les motifs que nous cherchons dans les sequences d'ADN peuvent contenir quelques variations ou mutations. Nous allons developpé dans cet partie differents algoritmes pour la recherche de motifs variables. 

1\. Comme dans le TME precedent, nous alons d'abord générer des données atificielles pour pouvoir tester les algorithmes. Faire une fonction pour générer `t` séquences artificielles de taille `n`. Implantez dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement.

In [65]:
import random

nuc = ('A', 'C', 'G', 'T')

k=4 #taille de motif
v=1 #nb de positions variable dans le motif
t=10 #longueur de sequences
n=3 #nb de sequence

def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n:int, upper=False):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront minuscule, False majuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    for i in range(n):
        sequence += random.choice(nuc) if upper else random.choice(nuc).lower()
        
    return sequence
 

def modifierMotif(motif:str, nbpos:int,  upper=False):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront minuscule, False majuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)
    for i in range(nbpos):
        motifM[random.randint(0,len(motifM)-1)]= random.choice(nuc)
    
    return "".join(motifM).upper() if upper else "".join(motifM).lower()
#tester modifMotif
print (modifierMotif("acg", 2))


def implantMotifVar(k, v, t, n):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : longueur de séquences 
    entrée n : nombre des séquences
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, 
    pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    
    motif = generateRandomSequence(k)
    
    return [insertMotif(generateRandomSequence(t - k),modifierMotif(motif, v), random.randint(0, t - k)) for i in range (n)]

adn = implantMotifVar(k, v, t, n)
print (adn)


ata
['tcatatacac', 'ttatatcaga', 'ttattcttag']


2\. Visualisation de motifs. Nous pouvons visualiser les motifs à l'aide des outils de LOGO https://weblogo.berkeley.edu/logo.cgi. Executer votre fonction de generation de motif variable, extraire les motifs et visulise-le à l'aide de webLogo. Utiliser les parametre ci-dessous.


In [66]:
k=8 #taille de motif
v=2 #nb de positions variable dans le motif
t=10 #nb de sequences
n=10 #longuer des sequence

adn = implantMotifVar(k, v, t, n)
for i in adn :
    print(i)

atgttcgcag
gtccccagaa
gtggccagca
tcgtgcccaa
gtacccagtg
ctgccctgga
ggttcccagc
gtgcccagac
cgtgcccaca
gagccaagta


3\. Implémentez l'algorithme _"Median String Search"_ pour chercher des motifs de taille variable. Tester le sur les jeux de données gnerer precedement. Vous devez éliminer les motifs peu complexe pour éviter les calculs inutiles. Pensez à eliminer tous les motifs ayant une formation diferents de motifs attendues, par exemple, les motifs ayant deux base repeté comme ACACACAC, etc.  

In [67]:
from itertools import product


def removeLowComplexe(motifs,k):
    validMotif = []
    double_consecutif=0
    for key in motifs:
        for i in range(0,len(key)-3):
            if (key[i] == key[i+2] and key[i+1] == key[i+3]):
                double_consecutif=0
            else:
                double_consecutif=1
                break
        if not(double_consecutif == 0 or ((key.lower()).count('a') > k) or  ((key.lower()).count('t') > k) or  ((key.lower()).count('c') > k) or  ((key.lower()).count('g') > k)):
            validMotif.append(key)
    return validMotif


                
#Genere tous les K-mers de taille K ayant de AAA... à TTT...

def kmerList(allkmers):
    return [''.join(i) for i in allkmers]
k=8
allkmers = product(nuc, repeat=k)
kmers = kmerList(allkmers)


print (len(kmers))
kmersValid = removeLowComplexe(kmers, 4)
print (len(kmersValid))
print (kmersValid[0])



65536
58368
AAAACCCC


In [68]:
k

8

In [69]:
import math

def hamdist(str1:str, str2:str):
    """
    Calcul la distance de hamming entre deux chaînes de caractères
    entrée str1: chaîne de caractères
    entrée str2: chaîne de caractères
    sortie distance: distance de hamming
    """
    diffs = 0
    for i in range(min(len(str1),len(str2))):
        if (str1[i] != str2[i]):
            diffs+=1

    return diffs



def totalDistance(motif:str, sequences, k):
    """
    Calcul la totalDistance
    entrée motif: sequence consensus
    entrée sequences:liste chaîne de caractères
    entrée k: taille du motif 
    entrée sequences: matrice de dimension txn contenant les séquences
    sortie totalDistance: somme de distance de hamming minimal
    """
    td = 0
    motif = motif.upper()
    for sequence in sequences : 
        sequence = sequence.upper()
        minimum = k+1
        for i in range(len(sequence)-k+1) :
            tmp = hamdist(sequence[i:i+k],motif)
            if minimum>tmp:
                minimum = tmp  
        td += minimum
    return td

def MedianStringSearch(allkmers, sequences, k):
    """
    Implement l'algorithme MedianStringSearch
    entrée allkmers: liste de K-mers valides
    entrée sequences: matrice de dimension txn contenant les séquences
    entrée k: taille du motif 
    sortie bestMotif: le motif que minimise les distances
    sortie bestDistance: la distance minimal
    sortie motifDist: un dictionnaire contenant les motifs et leurs distances
    """
    allkmers = removeLowComplexe(allkmers,5)
    bestDistance = math.inf
    bestMotif =allkmers[0]
    motifDist = {}
    for kmer in allkmers:
        tmp =  totalDistance(kmer, sequences, k)
        motifDist[kmer] = tmp
        if bestDistance>tmp:
            bestMotif = kmer
            bestDistance = tmp               
    return bestMotif, bestDistance, motifDist

MedianStringSearch(kmers, adn,k)


('GTGCCCAG',
 12,
 {'AAAAACCC': 60,
  'AAAAACCG': 59,
  'AAAAACCT': 62,
  'AAAAACGC': 59,
  'AAAAACGG': 59,
  'AAAAACGT': 63,
  'AAAAACTC': 60,
  'AAAAACTG': 58,
  'AAAAACTT': 64,
  'AAAAAGCC': 59,
  'AAAAAGCG': 60,
  'AAAAAGCT': 61,
  'AAAAAGGC': 58,
  'AAAAAGGG': 62,
  'AAAAAGGT': 62,
  'AAAAAGTC': 59,
  'AAAAAGTG': 61,
  'AAAAAGTT': 64,
  'AAAAATCC': 64,
  'AAAAATCG': 65,
  'AAAAATCT': 66,
  'AAAAATGC': 61,
  'AAAAATGG': 65,
  'AAAAATGT': 65,
  'AAAAATTC': 64,
  'AAAAATTG': 66,
  'AAAAATTT': 69,
  'AAAACACC': 56,
  'AAAACACG': 56,
  'AAAACACT': 57,
  'AAAACAGC': 51,
  'AAAACAGG': 52,
  'AAAACAGT': 52,
  'AAAACATC': 57,
  'AAAACATG': 58,
  'AAAACATT': 59,
  'AAAACCAC': 49,
  'AAAACCAG': 43,
  'AAAACCAT': 51,
  'AAAACCCA': 52,
  'AAAACCCC': 55,
  'AAAACCCG': 50,
  'AAAACCCT': 56,
  'AAAACCGA': 53,
  'AAAACCGC': 54,
  'AAAACCGG': 51,
  'AAAACCGT': 56,
  'AAAACCTA': 54,
  'AAAACCTC': 56,
  'AAAACCTG': 51,
  'AAAACCTT': 58,
  'AAAACGAC': 57,
  'AAAACGAG': 52,
  'AAAACGAT': 59,
  'AAAACGC

In [70]:
#Apliquez l'algoritme MedianStringSearch sur les sequences du brin complementaire

def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

s = []
for sequence in adn:
    s.append(reverseComplement(sequence.upper()))
MedianStringSearch(kmersValid, s, k)

('CTGGGCAC',
 12,
 {'AAAACCCC': 61,
  'AAAACCCG': 61,
  'AAAACCCT': 62,
  'AAAACCGC': 59,
  'AAAACCGG': 62,
  'AAAACCGT': 63,
  'AAAACCTC': 60,
  'AAAACCTG': 63,
  'AAAACCTT': 66,
  'AAAACGCC': 58,
  'AAAACGCG': 58,
  'AAAACGCT': 58,
  'AAAACGGC': 58,
  'AAAACGGG': 59,
  'AAAACGGT': 60,
  'AAAACGTC': 61,
  'AAAACGTG': 63,
  'AAAACGTT': 64,
  'AAAACTCC': 63,
  'AAAACTCG': 64,
  'AAAACTCT': 64,
  'AAAACTGC': 63,
  'AAAACTGG': 65,
  'AAAACTGT': 66,
  'AAAACTTC': 66,
  'AAAACTTG': 68,
  'AAAACTTT': 70,
  'AAAAGCCC': 54,
  'AAAAGCCG': 57,
  'AAAAGCCT': 56,
  'AAAAGCGC': 53,
  'AAAAGCGG': 57,
  'AAAAGCGT': 56,
  'AAAAGCTC': 53,
  'AAAAGCTG': 57,
  'AAAAGCTT': 58,
  'AAAAGGCC': 50,
  'AAAAGGCG': 52,
  'AAAAGGCT': 51,
  'AAAAGGGC': 49,
  'AAAAGGGG': 54,
  'AAAAGGGT': 52,
  'AAAAGGTC': 52,
  'AAAAGGTG': 57,
  'AAAAGGTT': 56,
  'AAAAGTCC': 56,
  'AAAAGTCG': 60,
  'AAAAGTCT': 59,
  'AAAAGTGC': 55,
  'AAAAGTGG': 62,
  'AAAAGTGT': 60,
  'AAAAGTTC': 57,
  'AAAAGTTG': 64,
  'AAAAGTTT': 64,
  'AAAATCC

## Partie B : Recherche de motifs variable sur vos données

Chercher sur le fichier "Sequence_by_Peaks_G*.fasta" contiens les regions de peak trouvé par ChipSeq, qui contient probablement le Facteur de Transcription que nous cherchons. Apliquer l'alogorithm Median Sting pour chercher les motifs dans votre fichier "Sequence_by_Peaks_G*. Il faut bien evidement enlever les motifs peu complexe.

In [71]:
k=10
feq=10
top= 50
genome = "Sequence_by_Peaks_3.fasta"

def searchMotifs(k:int, sequences:list):
    """
    Cherche les motifs de taille k dans un ensemble de séquences
    entrée k : taille du motif
    entrée séquences : liste de sequences
    sortie motifs: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    >>>searchMotifs(3, ['TAAGTAA', 'TATAA', 'CTATC'])
    {'TAA': 3, 'AAG': 1, 'AGT': 1, 'GTA': 1, 'TAT': 2, 'ATA': 1, 'CTA': 1, 'ATC': 1}
    """

    motifs  = {}
    
    for seq in sequences:
        for i in range(len(seq) - k + 1):
            if seq[i:(k + i)] in motifs:
                motifs[seq[i:(k + i)]] += 1
            else:
                motifs[seq[i:(k + i)]] = 1
                
    return motifs


def readFasta(fastaFileName):
    """
    Read a fasta file
    entrée fastaFileName: nom du fichier fasta
    sortie sequences: liste contenant toutes les sequences du fichier
    """
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())

    return sequence

sequences = readFasta(genome)

motifs = searchMotifs(k,sequences)

motifs = removeLowComplexe(motifs,k/2)


In [72]:
bestMotif, bestDistance, motifDist = MedianStringSearch(["TTTGCTCA"], motifs, 8)
print ("Motif evalué Mac1 :",bestMotif,"Best Distance:", bestDistance)
print("---------------------------------------------------------------")
bestMotif, bestDistance, motifDist = MedianStringSearch(["CTCATTGG","CTGATTGG"], motifs, 8)
print ("Motif evalué Hap5 :",bestMotif,"Best Distance:", bestDistance)
print("---------------------------------------------------------------")
bestMotif, bestDistance, motifDist = MedianStringSearch(["TGCACCC"], motifs, 7)
print ("Motif evalué Aft1 :",bestMotif,"Best Distance:", bestDistance)
print("---------------------------------------------------------------")
bestMotif, bestDistance, motifDist = MedianStringSearch(["TCCGCGGA"], motifs, 8)
print ("Motif evalué Pdr1 :",bestMotif,"Best Distance:", bestDistance)
bestMotif, bestDistance, motifDist = MedianStringSearch(["TCCGTGGA"], motifs, 8)
print ("Motif evalué Pdr1 :",bestMotif,"Best Distance:", bestDistance)
print("---------------------------------------------------------------")
bestMotif, bestDistance, motifDist = MedianStringSearch(["CAGCAA"], motifs, 6)
print ("Motif evalué ATM1 :",bestMotif,"Best Distance:", bestDistance)
bestMotif, bestDistance, motifDist = MedianStringSearch(["CAGCAAAA"], motifs, 8)
print ("Motif evalué ATM1 :",bestMotif,"Best Distance:", bestDistance)
print("---------------------------------------------------------------")
bestMotif, bestDistance, motifDist = MedianStringSearch(["CCTTNAGGGT"], motifs, 10)
print ("Motif evalué Zap :",bestMotif,"Best Distance:", bestDistance)
print("---------------------------------------------------------------")
bestMotif, bestDistance, motifDist = MedianStringSearch(["CGGATCCG"], motifs, 8)
print ("Motif evalué Sef1 :",bestMotif,"Best Distance:", bestDistance)

Motif evalué Mac1 : TTTGCTCA Best Distance: 157641
---------------------------------------------------------------
Motif evalué Hap5 : CTGATTGG Best Distance: 158196
---------------------------------------------------------------
Motif evalué Aft1 : TGCACCC Best Distance: 135352
---------------------------------------------------------------
Motif evalué Pdr1 : TCCGCGGA Best Distance: 167557
Motif evalué Pdr1 : TCCGTGGA Best Distance: 163101
---------------------------------------------------------------
Motif evalué ATM1 : CAGCAA Best Distance: 102871
Motif evalué ATM1 : CAGCAAAA Best Distance: 159307
---------------------------------------------------------------
Motif evalué Zap : CCTTNAGGGT Best Distance: 250688
---------------------------------------------------------------
Motif evalué Sef1 : CGGATCCG Best Distance: 164871
